In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(".")))

import pandas as pd
from utils import GCSHelper
from tqdm import tqdm
import re

gcs_helper = GCSHelper("../keys/gcs_key.json", bucket_name="maple_preprocessed_data")


/opt/ml/.cache/pypoetry/virtualenvs/maple-project-TvhCNGkp-py3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [117]:
# 기존 ~~색 제거에서 갈색을 제거하지 못하는 오류를 해결하기 위해, 1.0 버전까지 거슬러 올라가야함.
# 큰 설명 없이, 1.1 ~ 1.4의 내용을 대부분 가져오자. 

# 1.1 에 대해서는 약간 수정한다. 
# 기존 모든 아이템에 대해서 ~~색을 제거했던 것에서, 헤어에 대해서만 진행한다.
# 오류 방지를 위해 검정색~갈색을 모두 지정하여 입력한다. 

def ver1_5():

    dependent_file = "item_KMST_1149_VER1.0.csv"

    # 1.1 시작
    gcs_helper.change_bucket("maple_preprocessed_data")
    item_df = gcs_helper.read_df_from_gcs(blob_name=dependent_file)

    item_df["requiredJobs"] = item_df["requiredJobs"].replace("None", "['Beginner']")
    item_df["requiredLevel"] = item_df["requiredLevel"].replace("None", 0)

    item_df = item_df[item_df["name"] != "None"]
    item_df = item_df[item_df["name"] != "-"]

    pattern = r"[!@#$%^&*()_+-=`~,.<>/?{}\s\[\]]"
    item_df["name_processed"] = item_df["name"].apply(
        lambda x: re.sub(pattern, "", x.lower())
    )

    pattern = r"검은색|빨간색|주황색|노란색|초록색|파란색|보라색|갈색"
    new_names = []
    for i, row in item_df.iterrows():
        if row["subCategory"] != "Hair":
            new_names.append(row["name_processed"])
        else:
            new_names.append(re.sub(pattern, "", row["name_processed"]))

    item_df["name_processed"] = new_names
    item_df = item_df.drop_duplicates(["name_processed"]).reset_index(
        drop=True
    )

    item_df["requiredGender"] = item_df["requiredGender"].apply(
        lambda x: min(x, 2)
    )
    

    # 1.2 시작
    def equipCategory(x):
        if x in ["Hat", "Top", "Face", "Hair", "Overall", "Bottom", "Shoes"]:
            return x
        return "Weapon"

    item_df["equipCategory"] = item_df["subCategory"].apply(equipCategory)

    # 1.3 시작
    item_df["gcs_image_url"] = "https://storage.googleapis.com/maple_web/" + item_df["gcs_image_url"]

    # 1.4 시작
    item_df["isCash"] = item_df["isCash"].apply(int)

    rename_dict = {k:v for k, v in zip(item_df.columns,['required_jobs', 'required_level', 'required_gender', 'is_cash', 'desc',
        'item_id', 'name', 'overall_category', 'category', 'sub_category', 'low_item_id',
        'high_item_id', 'image_url', 'gcs_image_url', 'name_processed',
        'equip_category'])}
    item_df = item_df.rename(columns=rename_dict)
    
    # 1.5 독자적인 부분 추가
    
    # 학습 시 이미지를 로컬로 다운로드 해서 진행해야하기 때문에, image의 로컬 경로를 추가한다. 
    # 이미지 다운로드 경로는 modeling/data/image/item/123456.png 으로 고정이다. 
    item_df["local_image_path"] = "modeling/data/" + item_df["gcs_image_url"].apply(lambda x: x[41:])

    # iloc 접근 시 혼동이 없기 위해 명확한 index 명시
    item_df = item_df.reset_index()

    # 각 부위별 착용하지 않은 경우, 각각의 이미지가 개별적으로 필요함 (각 카테고리의 평균)
    # 해당 이미지를 위한 더미row 생성
    hat_row = [10093, "dummy", 0, 0, 0, "None", 1, "dummy_hat", "dummy", "dummy", "dummy", 0, 0, "dummy", "dummy", "dummy", "Hat", "modeling/data/image/item/1.png"]
    hair_row = [10094, "dummy", 0, 0, 0, "None", 2, "dummy_hair", "dummy", "dummy", "dummy", 0, 0, "dummy", "dummy", "dummy", "Hair", "modeling/data/image/item/2.png"]
    face_row = [10095, "dummy", 0, 0, 0, "None", 3, "dummy_face", "dummy", "dummy", "dummy", 0, 0, "dummy", "dummy", "dummy", "Face", "modeling/data/image/item/3.png"]
    overall_row = [10096, "dummy", 0, 0, 0, "None", 4, "dummy_overall", "dummy", "dummy", "dummy", 0, 0, "dummy", "dummy", "dummy", "Overall", "modeling/data/image/item/4.png"]
    top_row = [10097, "dummy", 0, 0, 0, "None", 5, "dummy_top", "dummy", "dummy", "dummy", 0, 0, "dummy", "dummy", "dummy", "Top", "modeling/data/image/item/5.png"]
    bottom_row = [10098, "dummy", 0, 0, 0, "None", 6, "dummy_bottom", "dummy", "dummy", "dummy", 0, 0, "dummy", "dummy", "dummy", "Bottom", "modeling/data/image/item/6.png"]
    shoes_row = [10099, "dummy", 0, 0, 0, "None", 7, "dummy_sheos", "dummy", "dummy", "dummy", 0, 0, "dummy", "dummy", "dummy", "Sheos", "modeling/data/image/item/7.png"]
    weapon_row = [10100, "dummy", 0, 0, 0, "None", 8, "dummy_weapon", "dummy", "dummy", "dummy", 0, 0, "dummy", "dummy", "dummy", "Weapon", "modeling/data/image/item/8.png"]

    dummy = pd.DataFrame([hat_row, hair_row, face_row, overall_row, top_row, bottom_row, shoes_row, weapon_row])
    dummy.columns = item_df.columns

    item_df = pd.concat([item_df, dummy], axis=0)

    return item_df

item_df = ver1_5()

gcs_helper.change_bucket("maple_preprocessed_data")
gcs_helper.upload_df_to_gcs("item_KMST_1149_VER1.5.csv", item_df)
gcs_helper.upload_df_to_gcs("item_KMST_1149_latest.csv", item_df)